In [27]:
import os, glob
from PyPDF2 import  PdfFileMerger, PdfFileReader
arcpy.env.overwriteOutput = True

In [29]:
# Project gdb and folder locations
projectGDB = r"C:\PythonBook\Chapter12\Chapter12.gdb"
projectFolder = "\\".join(projectGDB.split("\\")[:-1])

# bus and census data that already exists in the project gdb
busLines = os.path.join(projectGDB,"Summer21RouteShape")
censusPoly = os.path.join(projectGDB,"AlamedaContraCostaCounty_RaceHispanic_BlockGroup")
newBusLines = os.path.join(projectGDB,"AC_Transit_AdditionalTransbay")

# summary stat table to be created for the reference community calculations
sumStatTable = os.path.join(projectGDB,"SummStat_RaceHispanic_StudyArea")

# Feature layers created for selections
studyAreaFL = "SelectedCensus_FL"
busLinesFL = "SelectedBusLines_FL"
newBusCBGs = "NewBusCBGsRefComm"

# Suspended bus route and data created for suspended bus route
busRoute = "C"
newBusLinesSel = os.path.join(projectGDB,"NewTransbayLine_{0}".format(busRoute))
cbgStudyArea = os.path.join(projectGDB,"CBG_StudyArea_Bus_{0}".format(busRoute))

# Project and map and layout object for the map, inset map, and map frames for map and inset map
project = arcpy.mp.ArcGISProject("CURRENT")
m = project.listMaps("Map")[0]
inset = project.listMaps("Inset")[0]
layout = project.listLayouts("Layout")[0]
mf = layout.listElements("mapframe_element","Map Frame")[0]
mfInset = layout.listElements("mapframe_element","Inset")[0]

# table header, table box, title text elements, and legend element
tableHeader = layout.listElements("TEXT_ELEMENT","DetailsHeader")[0]
tableBox = layout.listElements("TEXT_ELEMENT","DetailsBox")[0]
title = layout.listElements("TEXT_ELEMENT","Title")[0]
legend = layout.listElements("LEGEND_ELEMENT","Legend")[0]


In [30]:
with arcpy.EnvManager(addOutputsToMap=False):
    arcpy.management.MakeFeatureLayer(censusPoly,studyAreaFL)
    arcpy.management.MakeFeatureLayer(busLines,busLinesFL,"PUB_RTE IN ('W', 'V', 'U', 'P', 'O', 'NX', 'NL', 'LA', 'L', 'J', 'G', 'F', 'DB1', 'DB')")
    arcpy.management.SelectLayerByLocation(studyAreaFL,"INTERSECT",busLinesFL,"0.5 Miles")
    summStats = []
    for field in arcpy.ListFields(studyAreaFL):
        if field.type == "Integer":
            print("Added {0} field name to summary stat list".format(field.name))
            summStats.append([field.name,"SUM"])
    arcpy.analysis.Statistics(studyAreaFL,sumStatTable,summStats)

Added total_pop field name to summary stat list
Added white field name to summary stat list
Added black field name to summary stat list
Added am_indian_nat_alaska field name to summary stat list
Added asian field name to summary stat list
Added nat_hawaiian_pac_island field name to summary stat list
Added some_other field name to summary stat list
Added two_or_more field name to summary stat list
Added hispanic_latino field name to summary stat list
Added total_minority field name to summary stat list


In [31]:
arcpy.management.AddField(sumStatTable,"MinorityRefComm_Prct","FLOAT")

for f in arcpy.ListFields(sumStatTable):
    print(f.name)
    if f.name.find("minority") != -1:
        print("Minority find value is {0} for field {1}".format(str(f.name.find("minority")),f.name))
        numerField = f.name
    if f.name.find("pop") != -1:
        print("Pop find value is {0} for field {1}".format(str(f.name.find("pop")),f.name))
        denomField = f.name
arcpy.management.CalculateField(sumStatTable,"MinorityRefComm_Prct","(!{0}!/!{1}!)*100".format(numerField,denomField))

with arcpy.da.SearchCursor(sumStatTable,["MinorityRefComm_Prct"]) as cursor:
    row = next(cursor)
    refCommPrct = row[0]

print("Reference Community Number is {0}".format(str(refCommPrct)))


OBJECTID
FREQUENCY
SUM_total_pop
Pop find value is 10 for field SUM_total_pop
SUM_white
SUM_black
SUM_am_indian_nat_alaska
SUM_asian
SUM_nat_hawaiian_pac_island
SUM_some_other
SUM_two_or_more
SUM_hispanic_latino
SUM_total_minority
Minority find value is 10 for field SUM_total_minority
MinorityRefComm_Prct
Reference Community Number is 65.56812286376953


In [32]:
with arcpy.EnvManager(addOutputsToMap=False):
    arcpy.analysis.Select(newBusLines,newBusLinesSel,"route_s_nm = '{0}'".format(busRoute))
    arcpy.management.MakeFeatureLayer(censusPoly,newBusCBGs)
    arcpy.management.SelectLayerByLocation(newBusCBGs,"INTERSECT",newBusLinesSel,"0.5 Miles")
    arcpy.management.SelectLayerByAttribute(newBusCBGs,"REMOVE_FROM_SELECTION","total_pop = 0")
    minorityGEOIDs = []
    allGEOIDs = []
    with arcpy.da.SearchCursor(newBusCBGs,["GEOID","percent_minority"]) as cursor:
        for row in cursor:
            allGEOIDs.append(row[0])
            if row[1] >= refCommPrct:
                minorityGEOIDs.append(row[0])
                print("Added {0} to minority GEOID list".format(row[0]))

    cbgStudyAreaTuple = tuple(allGEOIDs)
    cgbStudyAreaSQL = "GEOID in {0}".format(cbgStudyAreaTuple)
    print(cgbStudyAreaSQL)
    arcpy.analysis.Select(censusPoly,cbgStudyArea,cgbStudyAreaSQL)


Added 060014036001 to minority GEOID list
Added 060014013001 to minority GEOID list
Added 060014015001 to minority GEOID list
Added 060014014001 to minority GEOID list
Added 060014014002 to minority GEOID list
Added 060014016001 to minority GEOID list
Added 060014017003 to minority GEOID list
Added 060014010001 to minority GEOID list
Added 060014010002 to minority GEOID list
Added 060014010004 to minority GEOID list
Added 060014011003 to minority GEOID list
Added 060014251041 to minority GEOID list
Added 060014035013 to minority GEOID list
Added 060014251031 to minority GEOID list
Added 060014251022 to minority GEOID list
Added 060014251023 to minority GEOID list
GEOID in ('060014036001', '060014012003', '060014011004', '060014012001', '060014012002', '060014013001', '060014015001', '060014014001', '060014014002', '060014015002', '060014015003', '060014016001', '060014042002', '060014039001', '060014039003', '060014040001', '060014040002', '060014017001', '060014017003', '060014261004'

In [33]:
m.addDataFromPath(cbgStudyArea)
cbgStudyAreaLyr = m.listLayers("CBG_StudyArea_Bus_{0}".format(busRoute))[0]
cbgStudyAreaLyrSym = cbgStudyAreaLyr.symbology
cbgStudyAreaLyrSym.updateRenderer('GraduatedColorsRenderer')
cbgStudyAreaLyrSym.renderer.classificationField = "percent_minority"
cbgStudyAreaLyrSym.renderer.breakCount = 2
breakValue = refCommPrct
firstVal = 0
for brk in cbgStudyAreaLyrSym.renderer.classBreaks:
    brk.upperBound = breakValue
    if firstVal == 0:
        brk.label = "Minority Population > Reference Community"
        brk.symbol.color = {'RGB' : [255, 0, 0, 30]}
        brk.symbol.outlineColor = {'RGB' : [255, 0, 0, 50]}
    else:
        brk.label = "Minority Population < Reference Community"
        brk.symbol.color = {'RGB' : [0, 255, 0, 30]}
        brk.symbol.outlineColor = {'RGB' : [0, 255, 0, 50]}
    breakValue = 100
    firstVal += 1
cbgStudyAreaLyr.symbology = cbgStudyAreaLyrSym


In [34]:
m.addDataFromPath(newBusLinesSel)
newBusLyr = m.listLayers("NewTransbayLine_{0}".format(busRoute))[0]
newBusLyrSym = newBusLyr.symbology
newBusLyrSym.renderer.symbol.color = {"RGB" : [169, 0, 230, 100]}
newBusLyrSym.renderer.symbol.size = 1.5
newBusLyr.symbology = newBusLyrSym
newBusLyr.name = "Transbay Bus Route"


In [35]:
curInsetLayers = [l.name for l in inset.listLayers()]
if cbgStudyAreaLyr.name in curInsetLayers:
    insetCBGLyr = inset.listLayers(cbgStudyAreaLyr.name)[0]
    inset.removeLayer(insetCBGLyr)
    print("Removing old CBG Study Area Layer to Inset")
    inset.addLayer(cbgStudyAreaLyr)
    print("Adding new CBG Study Area Layer to Inset")
else:
    inset.addLayer(cbgStudyAreaLyr)
    print("Adding new CBG Study Area Layer to Inset")
if newBusLyr.name in curInsetLayers:
    insetBusLyr = inset.listLayers(newBusLyr.name)[0]
    inset.removeLayer(insetBusLyr)
    print("Removing old Transbay Bus Route to Inset")
    inset.addLayer(newBusLyr)
    print("Adding new Transbay Bus Route to Inset")
else:
    inset.addLayer(newBusLyr)
    print("Adding new Transbay Bus Route to Inset")


Adding new CBG Study Area Layer to Inset
Adding new Transbay Bus Route to Inset


In [36]:
insetStudyAreaLayer = inset.listLayers("CBG_StudyArea_Bus_{0}".format(busRoute))[0]
insetExtent = mfInset.getLayerExtent(insetStudyAreaLayer,False,True)
print(insetExtent)
mfInset.camera.setExtent(insetExtent)
mfInset.camera.scale = round((mfInset.camera.scale + 2000),0)
print(mfInset.camera.scale)


-122.347072470221 37.8072665294239 -122.211846529491 37.8503534704394 NaN NaN NaN NaN
441507.0


In [37]:
cbgLayerLabels = cbgStudyAreaLyr.listLabelClasses()[0]
cbgLayerLabels.expression = "$feature.GEOID"
cbgLayerLabels.visible = True
cbgStudyAreaLyr.showLabels = True

In [38]:
legendItemNames = [item.name for item in legend  .items]
if cbgStudyAreaLyr.name not in legendItemNames:
    legend.addItem(cbgStudyAreaLyr,"TOP")
if newBusLyr.name not in legendItemNames:
    legend.addItem(newBusLyr,"TOP")


In [39]:
for item in legend.items:
    if item.name in (cbgStudyAreaLyr.name,newBusLyr.name,"AlamedaContraCostaCounty_RaceHispanic_BlockGroup"):
        print('{0} is displayed in the legend'.format(item.name))
        item.visible = True   
    else:
        item.visible = False 


Transbay Bus Route is displayed in the legend
CBG_StudyArea_Bus_C is displayed in the legend
AlamedaContraCostaCounty_RaceHispanic_BlockGroup is displayed in the legend


In [40]:
prcField = []
prcDataDict = {}
for field in arcpy.ListFields(censusPoly):
    if field.type == "Single":
        print("Added {0} field name to prcField list".format(field.name))
        prcDataDict[field.name] = field.aliasName
        prcField.append(field.name)
tableHeader.text = "Percent Race/Hispanic"


Added prct_white field name to prcField list
Added prct_black field name to prcField list
Added prct_am_indian_nat_alaska field name to prcField list
Added prct_asian field name to prcField list
Added prct_nat_hawiian_pac_island field name to prcField list
Added prct_some_other field name to prcField list
Added prct_two_or_more field name to prcField list
Added prct_hispanic_latino field name to prcField list
Added percent_minority field name to prcField list


In [41]:
cbgLayer = m.listLayers("AlamedaContraCostaCounty_RaceHispanic_BlockGroup")[0]
i = 1
for geoid in minorityGEOIDs:
    print(i)
    print(geoid)
    cbgStudyAreaLyr.definitionQuery = "GEOID <> '{0}'".format(geoid)
    cbgLayer.definitionQuery = "GEOID = '{0}'".format(geoid)
    textBox = []
    with arcpy.da.SearchCursor(censusPoly,prcField,"GEOID = '{0}'".format(geoid)) as cursor2:
        row = next(cursor2)
        j=0
        while j < len(row):
            print(cursor2.fields[j])
            print(row[j])
            if j != (len(row)-1):
                tempVal = "{0}: {1}\r\n".format(prcDataDict[cursor2.fields[j]],round(row[j],2))
                print(tempVal)
            else:
                print("the last value")
                tempVal = "{0}: {1}".format(prcDataDict[cursor2.fields[j]],round(row[j],2))
                print(tempVal)
            textBox.append(tempVal)
            j+=1
            
    tableBox.text = ""
    for text in textBox:
        tableBox.text += text
        print(text) 
    tableBox.height = 1.23
    tableBox.elementPositionY = 2.2813
        
    title.text = "Figure 2.{0}\r\nBus Route: {1}\r\nBlock Group: {2}".format(str(i),busRoute,str(geoid))
    print(title.text)
    cbgLayer.name = "Population in Block Group {0}".format(geoid) 
    
    # If scale is less than 10000 set to 10000, otherwise add 2000 and round to a whole number 
    extent = mf.getLayerExtent(cbgLayer,False,True)
    print(extent)
    mf.camera.setExtent(extent)
    print(mf.camera.scale)
    if mf.camera.scale < 10000:
        mf.camera.scale = 10000
    else:
        mf.camera.scale = round((mf.camera.scale + 2000),0)
    print(mf.camera.scale)
    pdfFigNum = str(i)
    if len(pdfFigNum) == 1:
        pdfFigNum = "0"+pdfFigNum
    layout.exportToPDF(os.path.join(projectFolder,"Figure_2_{0}_BusRoute_{1}_GEOID_{2}.pdf".format(pdfFigNum,busRoute,geoid)))
    cbgLayer.name = "AlamedaContraCostaCounty_RaceHispanic_BlockGroup" 
    i+=1
    

1
060014036001
prct_white
13.220000267028809
Percent White: 13.22

prct_black
64.58000183105469
Percent Black: 64.58

prct_am_indian_nat_alaska
0.0
Percent American Indian/Native Alaskan: 0.0

prct_asian
5.679999828338623
Percent Asian: 5.68

prct_nat_hawiian_pac_island
0.0
Percent Native Hawaiian/Pacific Islander: 0.0

prct_some_other
0.0
Percent Some Other Race: 0.0

prct_two_or_more
5.679999828338623
Percent Two Or More Races: 5.68

prct_hispanic_latino
10.829999923706055
Percent Hispanic/Latino: 10.83

percent_minority
86.77999877929688
the last value
Percent Minority: 86.78
Percent White: 13.22

Percent Black: 64.58

Percent American Indian/Native Alaskan: 0.0

Percent Asian: 5.68

Percent Native Hawaiian/Pacific Islander: 0.0

Percent Some Other Race: 0.0

Percent Two Or More Races: 5.68

Percent Hispanic/Latino: 10.83

Percent Minority: 86.78
Figure 2.1
Bus Route: C
Block Group: 060014036001
-122.254026999746 37.810504999806 -122.248008000069 37.8190010003959 NaN NaN NaN NaN
744

Block Group: 060014251023
-122.29678700032 37.8301069999382 -122.291015000342 37.8389029999735 NaN NaN NaN NaN
7709.970432715083
10000.0


In [42]:
pdfMergeObj = PdfFileMerger()

pdfFiles = glob.glob(os.path.join(projectFolder,"*BusRoute_{0}*.pdf".format(busRoute)))
for pdf in pdfFiles:
    print(pdf)
    pdfMergeObj.append(PdfFileReader(pdf,"rb"))

pdfMergeObj.write(os.path.join(projectFolder,"Figure_2_BusRoute_{0}_MinorityRace_Greater_RefComm.pdf".format(busRoute)))


C:\PythonBook\_TestUploadedData\Chapter12\Figure_2_01_BusRoute_C_GEOID_060014036001.pdf
C:\PythonBook\_TestUploadedData\Chapter12\Figure_2_02_BusRoute_C_GEOID_060014013001.pdf
C:\PythonBook\_TestUploadedData\Chapter12\Figure_2_03_BusRoute_C_GEOID_060014015001.pdf
C:\PythonBook\_TestUploadedData\Chapter12\Figure_2_04_BusRoute_C_GEOID_060014014001.pdf
C:\PythonBook\_TestUploadedData\Chapter12\Figure_2_05_BusRoute_C_GEOID_060014014002.pdf
C:\PythonBook\_TestUploadedData\Chapter12\Figure_2_06_BusRoute_C_GEOID_060014016001.pdf
C:\PythonBook\_TestUploadedData\Chapter12\Figure_2_07_BusRoute_C_GEOID_060014017003.pdf
C:\PythonBook\_TestUploadedData\Chapter12\Figure_2_08_BusRoute_C_GEOID_060014010001.pdf
C:\PythonBook\_TestUploadedData\Chapter12\Figure_2_09_BusRoute_C_GEOID_060014010002.pdf
C:\PythonBook\_TestUploadedData\Chapter12\Figure_2_10_BusRoute_C_GEOID_060014010004.pdf
C:\PythonBook\_TestUploadedData\Chapter12\Figure_2_11_BusRoute_C_GEOID_060014011003.pdf
C:\PythonBook\_TestUploadedData\

In [43]:
m.removeLayer(newBusLyr)
m.removeLayer(cbgStudyAreaLyr)
